In [1]:
import os

import numpy as np
import pandas as pd

from augmentation import Augmentation, Image
from config import ROOT_PATH

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense

from concurrent.futures import ThreadPoolExecutor, wait

SHAPE = (120, 120, 3)
METHODS = ["rotation", "shifting", "blurring"]

In [2]:
def nn_output(elem, max_elem: int=8):
    res = np.zeros(max_elem, dtype=int)
    res[elem - 1] = 1
    return res

In [3]:
def add_data(label):
    global data
    dir_name = os.path.join(os.path.join(ROOT_PATH, "data"), str(label))
    for file in os.listdir(dir_name):
        image = Image(filename=os.path.join(dir_name, file))
        image = image.shearing(tuple(SHAPE[:2]))
        data.append((image.image_matrix / 255, nn_output(label)))
        aug = Augmentation(image, count=29, methods_list=METHODS)
        aug.create_children()
        for child in aug.children:
            data.append((child.image_matrix / 255,  nn_output(label)))

In [4]:
def mixup():
    global X, y, X_, y_
    index_1 = np.random.choice(len(X))
    index_2 = np.random.choice(len(X))

    x_1, label_1 = X[index_1], y[index_1]
    x_2, label_2 = X[index_2], y[index_2]

    alpha = np.random.uniform(0.1, 0.2)

    X_.append(x_1 * alpha + x_2 * (1 - alpha))
    y_.append(label_1 * alpha + label_2 * (1 - alpha))

In [5]:
data = []
with ThreadPoolExecutor(max_workers=8) as executor:
    preds = [executor.submit(add_data, label) for label in range(1, 9)]
    wait(preds)
X = [i[0] for i in data]
y = [i[1] for i in data]    

X_ = []
y_ = []

with ThreadPoolExecutor(max_workers=16) as executor:
    preds = [executor.submit(mixup) for label in range(60000)]
    wait(preds)

In [6]:
X = np.array(X + X_)
y = np.array(y + y_)

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, AvgPool2D
from tensorflow.keras.datasets import mnist

In [8]:
model = load_model("my_MNIST.h5")
for layer in model.layers[:-5]:
    layer.trainable = False
model.pop()
model.add(Dense(8, kernel_initializer="normal", activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 6)       456       
_________________________________________________________________
average_pooling2d (AveragePo (None, 60, 60, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 58, 10)        550       
_________________________________________________________________
average_pooling2d_1 (Average (None, 29, 29, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 17)        4267      
_________________________________________________________________
average_pooling2d_2 (Average (None, 12, 12, 17)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 27)          11502     
__________

model = Sequential()
model.add(Conv2D(8, (5, 5), input_shape = (32, 32, 3), activation = "relu"))
model.add(AvgPool2D(pool_size = (2, 2), strides=(2, 2)))
model.add(Conv2D(16, (5, 5), activation = "relu"))
model.add(AvgPool2D(pool_size = (2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(activation = "relu", units=84))
model.add(Dense(8, kernel_initializer="normal", activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [ ]:
model.fit(X, y, batch_size=512, epochs=100, verbose=1)

Epoch 1/100
66240/66240 [==============================] - 83s 1ms/step - loss: 2.0679 - acc: 0.3202
Epoch 2/100
66240/66240 [==============================] - 84s 1ms/step - loss: 1.4363 - acc: 0.6142
Epoch 3/100
66240/66240 [==============================] - 83s 1ms/step - loss: 1.2922 - acc: 0.7091
Epoch 4/100
66240/66240 [==============================] - 89s 1ms/step - loss: 1.2235 - acc: 0.7530
Epoch 5/100
66240/66240 [==============================] - 85s 1ms/step - loss: 1.1782 - acc: 0.7792
Epoch 6/100
66240/66240 [==============================] - 86s 1ms/step - loss: 1.1422 - acc: 0.7977
Epoch 7/100
66240/66240 [==============================] - 86s 1ms/step - loss: 1.1157 - acc: 0.8110
Epoch 8/100
35840/66240 [===============>..............] - ETA: 41s - loss: 1.0898 - acc: 0.8198

In [ ]:
model.save("final_model.h5")